In [ ]:
pip install yfinance

In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
import statsmodels.api as sm

In [ ]:
# Carico i dati Fama-French
path = 'F-F_Research_Data_Factors_weekly 2.csv'
factors_raw = pd.read_csv(path, skiprows=3)
factors = factors_raw.iloc[:, :5]
factors.columns = ['Date', 'Mkt-RF', 'SMB', 'HML', 'RF']
factors['Date'] = pd.to_datetime(factors['Date'], format='%d/%m/%Y', errors='coerce')
factors = factors.dropna(subset=['Date'])
factors = factors.sort_values('Date')
factors = factors.set_index('Date')
factors_monthly = factors.resample('M').last()
factors_monthly['YearMonth'] = factors_monthly.index.to_period('M')

<ipython-input-37-6140bf395682>:10: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  factors_monthly = factors.resample('M').last()


Analisi 4 azioni USA selezionate : Aflac Inc. (AFL) nel il settore assicurativo, Exxon Mobil (XOM), colosso del settore energetico, Intel (INTC), azienda leader nella produzione di semiconduttori. Infine, Southwest Airlines (LUV), compagnia aerea

In [ ]:
# Scarico dati azioni
tickers = ['AFL', 'XOM', 'INTC', 'LUV']
prezzi = yf.download(tickers, start='2019-12-20', end='2024-12-27', interval='1mo')['Close']
prezzi = prezzi.dropna(how='all')  # Rimuovo righe vuote se presenti

# Calcolo i rendimenti
rendimenti = prezzi.pct_change().dropna() * 100
rendimenti.index = pd.to_datetime(rendimenti.index)
rendimenti['YearMonth'] = rendimenti.index.to_period('M')
rendimenti.head()

[*********************100%***********************]  4 of 4 completed


Ticker,AFL,INTC,LUV,XOM,YearMonth
Date,,,,,
2020-02-01,-16.909062,-13.154999,-15.987622,-17.192542,2020-02
2020-03-01,-19.667412,-2.041591,-22.905407,-25.126136,2020-03
2020-04-01,8.761696,10.827794,-11.905994,22.386108,2020-04
2020-05-01,-2.067672,4.918315,2.719993,-2.151919,2020-05
2020-06-01,-0.403968,-4.389068,6.479753,0.257507,2020-06


In [ ]:
# Allineamento mesi
mesi_comuni = list(set(rendimenti['YearMonth']).intersection(set(factors_monthly['YearMonth'])))
rendimenti_final = rendimenti[rendimenti['YearMonth'].isin(mesi_comuni)].set_index('YearMonth')
factors_final = factors_monthly[factors_monthly['YearMonth'].isin(mesi_comuni)].set_index('YearMonth')
rendimenti_final = rendimenti_final.groupby(rendimenti_final.index).mean()

# Calcolo rendimenti in eccesso
risk_free = factors_final['RF']
rendimenti_excess = rendimenti_final[tickers].subtract(risk_free, axis=0)
rendimenti_excess.head()

Ticker,AFL,XOM,INTC,LUV
YearMonth,,,,
2020-02,-16.939062,-17.222542,-13.184999,-16.017622
2020-03,-19.700412,-25.159136,-2.074591,-22.938407
2020-04,8.760696,22.385108,10.826794,-11.906994
2020-05,-2.069672,-2.153919,4.916315,2.717993
2020-06,-0.406968,0.254507,-4.392068,6.476753


In [ ]:
# Dati per la regressione
X = factors_final[['Mkt-RF', 'SMB', 'HML']]
X = sm.add_constant(X)

# Dizionario per memorizzare i risultati
risultati = {}

# Regresione OLS per ogni azione
for azione in rendimenti_excess.columns:
    Y = rendimenti_excess[azione]
    modello = sm.OLS(Y, X).fit()
    risultati[azione] = {
        'beta': modello.params,
        'pvalue': modello.pvalues
    }

# DataFrame per i beta
beta_df = pd.DataFrame({k: v['beta'] for k, v in risultati.items()}).T

# DataFrame per i p-value
pval_df = pd.DataFrame({k: v['pvalue'] for k, v in risultati.items()}).T

# Risultati
def evidenzia_significativi(val):
    return 'background-color: green' if val < 0.05 else ''

# Stampa dei beta
print("Beta (coefficients):")
display(beta_df)

# Stampa dei p-value e evidenza dei significativi
print("\nP-Values:")
display(pval_df.style.applymap(evidenzia_significativi))


Beta (coefficients):


,const,Mkt-RF,SMB,HML
AFL,1.420867,0.375473,0.838869,1.148053
XOM,1.532331,0.365758,1.314691,1.622567
INTC,-1.546131,0.729216,0.553233,1.053008
LUV,-0.642701,0.021778,-1.460415,1.701599



P-Values:


<ipython-input-40-07ed9e8595d6>:33: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  display(pval_df.style.applymap(evidenzia_significativi))


,const,Mkt-RF,SMB,HML
AFL,0.163716,0.248624,0.255023,0.041898
XOM,0.220256,0.358892,0.147707,0.019899
INTC,0.295692,0.125303,0.604175,0.194528
LUV,0.650184,0.961650,0.158327,0.031645


In [ ]:
from scipy.optimize import linprog

In [ ]:
# Estrazione beta significativi (p-value < 0.05)
beta_sign = beta_df.where(pval_df < 0.05)

# Costruzione del sistema: 3 equazioni, 4 incognite
azioni = beta_sign.index.tolist()  # ['AFL', 'XOM', 'INTC', 'LUV']

# Righe: somma pesi, MKT-RF, HML
A_eq = []
b_eq = []

# Eq. 1: somma pesi = 1
A_eq.append([1]*4)
b_eq.append(1)

# Eq. 2: somma pesata beta MKT-RF = 0 (ignora NaN)
mkt_row = beta_sign['Mkt-RF'].values
A_eq.append(np.nan_to_num(mkt_row, nan=0.0).tolist())
b_eq.append(0)

# Eq. 3: somma pesata beta HML = 0
hml_row = beta_sign['HML'].values
A_eq.append(np.nan_to_num(hml_row, nan=0.0).tolist())
b_eq.append(0)

A_eq = np.array(A_eq)
b_eq = np.array(b_eq)

# Risoluzione del sistema (uso minimi quadrati)
pesi, _, _, _ = np.linalg.lstsq(A_eq, b_eq, rcond=None)
pesi = pesi / pesi.sum()  # normalizza per assicurare somma 1

# Visualizzazione pesi
pesi_portafoglio = pd.Series(pesi, index=azioni)
print("Pesi ottimizzati:\n", pesi_portafoglio)

# Calcolo rendimento portafoglio
rendimenti_portafoglio = rendimenti_excess[azioni].dot(pesi_portafoglio)

# T-statistica
media = rendimenti_portafoglio.mean()
dev_std = rendimenti_portafoglio.std()
t_stat = media / (dev_std / np.sqrt(60))

print(f"\nT-stat: {t_stat:.4f}")
if -1.96 < t_stat < 1.96:
    print("➤ Nessuna opportunità di arbitraggio (t-stat non significativo).")
else:
    print("➤ POSSIBILE arbitraggio (t-stat significativo!).")

Pesi ottimizzati:
 AFL     0.231831
XOM    -0.055564
INTC    0.927164
LUV    -0.103431
dtype: float64

T-stat: -0.5592
➤ Nessuna opportunità di arbitraggio (t-stat non significativo).
